## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,71.9167,114.0833,-29.67,100,85,4.41,Clouds : overcast clouds
1,1,Upernavik,GL,72.7868,-56.1549,0.43,98,100,4.56,Clouds : overcast clouds
2,2,Butaritari,KI,3.0707,172.7902,82.38,69,14,18.14,Clouds : few clouds
3,3,Iskateley,RU,67.6803,53.1512,-7.87,98,86,9.55,Clouds : overcast clouds
4,4,Wattegama,LK,6.7989,81.4808,69.58,91,6,5.35,Clear : clear sky


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Butaritari,KI,3.0707,172.7902,82.38,69,14,18.14,Clouds : few clouds
9,9,Hermanus,ZA,-34.4187,19.2345,85.89,64,62,20.15,Clouds : broken clouds
13,13,Cayenne,GF,4.9333,-52.3333,86.04,81,75,5.99,Clouds : broken clouds
14,14,Vaini,TO,-21.2000,-175.2000,75.36,100,100,13.80,Rain : light rain
15,15,Carnarvon,AU,-24.8667,113.6333,80.67,83,24,5.75,Clouds : few clouds
23,23,Puerto Ayora,EC,-0.7393,-90.3518,85.96,88,54,4.00,Clouds : broken clouds
25,25,Pochutla,MX,15.7432,-96.4661,85.82,34,0,6.76,Clear : clear sky
27,27,Chuy,UY,-33.6971,-53.4616,83.12,50,83,8.52,Clouds : broken clouds
28,28,Atuona,PF,-9.8000,-139.0333,78.69,74,13,21.97,Clouds : few clouds
38,38,Rikitea,PF,-23.1203,-134.9692,78.62,73,53,12.82,Clouds : broken clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                173
City                   173
Country                173
Lat                    173
Lng                    173
Max Temp               173
Humidity               173
Cloudiness             173
Wind Speed             173
Current Description    173
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                173
City                   173
Country                173
Lat                    173
Lng                    173
Max Temp               173
Humidity               173
Cloudiness             173
Wind Speed             173
Current Description    173
dtype: int64

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.38,Clouds : few clouds,3.0707,172.7902,
9,Hermanus,ZA,85.89,Clouds : broken clouds,-34.4187,19.2345,
13,Cayenne,GF,86.04,Clouds : broken clouds,4.9333,-52.3333,
14,Vaini,TO,75.36,Rain : light rain,-21.2000,-175.2000,
15,Carnarvon,AU,80.67,Clouds : few clouds,-24.8667,113.6333,
23,Puerto Ayora,EC,85.96,Clouds : broken clouds,-0.7393,-90.3518,
25,Pochutla,MX,85.82,Clear : clear sky,15.7432,-96.4661,
27,Chuy,UY,83.12,Clouds : broken clouds,-33.6971,-53.4616,
28,Atuona,PF,78.69,Clouds : few clouds,-9.8000,-139.0333,
38,Rikitea,PF,78.62,Clouds : broken clouds,-23.1203,-134.9692,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
    
hotel_df.count()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [87]:
# 7. Drop the rows where there is no Hotel Name.
for index, row in hotel_df.iterrows():
    if hotel_df.loc[index, "Hotel Name"] == "":
        hotel_df.drop(index, inplace=True)

In [90]:
## Check layout of Hotel table before saving to CSV
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.38,Clouds : few clouds,3.0707,172.7902,Isles Sunset Lodge
9,Hermanus,ZA,85.89,Clouds : broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
13,Cayenne,GF,86.04,Clouds : broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
14,Vaini,TO,75.36,Rain : light rain,-21.2000,-175.2000,Keleti Beach Resort
15,Carnarvon,AU,80.67,Clouds : few clouds,-24.8667,113.6333,Hospitality Carnarvon


In [92]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [95]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Currently</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [96]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))